In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime as dt
pd.set_option('future.no_silent_downcasting', True)

# Get WACC

In [4]:
def get_risk_free_rate():
    """
    Get the current 10-year Treasury yield (risk-free rate) from an API.
    """
    try:
        tnx = yf.Ticker("^TNX")
        current_yield = tnx.history(period="1d")['Close'].iloc[-1] * 0.01
        return round(current_yield, 3)
    except:
        return 0.04
        
def get_wacc(ticker):
    stock= yf.Ticker(ticker)
    #Get Market Cap
    market_cap = stock.info.get('marketCap', 0)

    #Get Total Debt
    total_debt = stock.balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in stock.balance_sheet else 0

    # Get Beta
    beta = stock.info.get('beta', 1)

    # Get Interest Expense & Total Revenue to estimate Cost of Debt (Rd)
    interest_expense = stock.financials.loc['Interest Expense'].iloc[0] if 'Interest Expense' in stock.financials else 0
    cost_of_debt = abs(interest_expense / total_debt) if total_debt else 0.05  # Default 5% if no data

    # Get Tax Rate
    income_tax = stock.financials.loc['Income Tax Expense'].iloc[0] if 'Income Tax Expense' in stock.financials else 0
    ebit = stock.financials.loc['EBIT'][0] if 'EBIT' in stock.financials else 0
    tax_rate = income_tax / ebit if ebit else 0.21  # Default to 21%

    # Get Risk-Free Rate and Market Return (estimated ~8%)
    risk_free_rate = get_risk_free_rate()
    market_return = 0.08

    #Calculate Cost of Equity (Re) using CAPM
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
    
    # Total Value (V = E + D)
    total_value = market_cap + total_debt

    # Calculate WACC
    wacc = (market_cap / total_value * cost_of_equity) + (total_debt / total_value * cost_of_debt * (1 - tax_rate))

    return round(wacc, 4) 
    
#past 4 year revenue growth
def get_revenue_growth(ticker):
    stock= yf.Ticker(ticker)
    revenue= stock.income_stmt.loc['Total Revenue'].dropna()
    revenue = revenue.sort_index()

    growth_rates = revenue.pct_change()
    avg_growth = pd.Series(growth_rates[1:]).mean()

    return avg_growth
    
#5 year revenue projection
def get_revenue_projection(ticker, growth_rate):
    stock= yf.Ticker(ticker)
    revenue_projection = []
    projection_years=[]
    last_year = stock.income_stmt.columns[0].year
    last_year_revenue= stock.income_stmt.loc['Total Revenue'].iloc[0]/1e9
    
    for x in range(5):
        next_year_revenue = (last_year_revenue * (1 + growth_rate))
        revenue_projection.append(round(next_year_revenue, 2))
        last_year_revenue = next_year_revenue
    
        projection_years.append(last_year + (x + 1))
        
    return pd.Series(revenue_projection, projection_years) 

def get_ebit_margin(ticker):
    stock= yf.Ticker(ticker)
    if 'Operating Income' in stock.income_stmt.index:
        operating_income = stock.income_stmt.loc['Operating Income'].dropna()
        revenue = stock.income_stmt.loc['Total Revenue'].dropna()
        operating_margin = operating_income/ revenue
        
        return operating_margin.mean()
        
    else:
        print("Operating Income not found")
        
def get_ebit_projection(revenue_projection, ebit_margin):
    ebit_projection = []
    
    for revenue in revenue_projection:
        ebit = revenue * ebit_margin
        ebit_projection.append(round(ebit, 2))
    return pd.Series(ebit_projection, revenue_projection.index) 

def get_NOPAT(ebit_projection):
    ebit_after_tax = []
    tax_rate= 0.21

    for ebit in ebit_projection:
        nopat = ebit * (1 - tax_rate)
        ebit_after_tax.append(nopat)
    return pd.Series(ebit_after_tax, ebit_projection.index) 
    
def get_depreciation_and_amortization(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Depreciation And Amortization' in stock.cashflow.index:
        da = stock.cashflow.loc['Depreciation And Amortization'].dropna()/1e9
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        da_rate = (da/revenue).mean()
        return da_rate * projection
    else:
        print("Depreciation & Amortization not found")

def get_capex(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Capital Expenditure' in stock.cashflow.index:
        capex = abs(stock.cashflow.loc['Capital Expenditure'].dropna()/1e9)
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        capex_rate = (capex/revenue).mean()
        return capex_rate * projection
    else:
        print("Capital Expenditure not found")

def get_workingcapital(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Change In Working Capital' in stock.cashflow.index:
        nwc = abs(stock.cashflow.loc['Change In Working Capital'].dropna()/1e9)
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        nwc_rate = (nwc/revenue).mean()
        return nwc_rate * projection
    else:
        print("Change In Working Capital not found")
        
def get_fcf(NOPAT, da, capex, nwc):
    fcf = NOPAT + da - capex - nwc
    
    return fcf

def get_fcf_pv(fcf_projection, wacc):
    discounted = []
    
    for t, fcf in enumerate(fcf_projection, start=1):
        pv = fcf/((1 + wacc) ** t)
        discounted.append(pv)
        
    return pd.Series(discounted, fcf_projection.index)  

def get_terminal_value(fcf_projection, wacc):
    terminal_growth = 0.02
    final_year_fcf = fcf_projection.iloc[-1]
    
    terminal_value = (final_year_fcf * (1 + terminal_growth))/(wacc - terminal_growth)
    
    return terminal_value

def get_pv_tv(tv, wacc):
    n = 5
    pv_tv= tv/((1 + wacc) ** n)

    return pv_tv

def get_enterprise_value(pv_fcf, pv_tv):
    pv_fcf_sum = pv_fcf.sum()
    
    ev = pv_fcf_sum + pv_tv

    return ev
def get_net_debt(ticker):  
    stock= yf.Ticker(ticker)
    cash = stock.balance_sheet.loc['Cash And Cash Equivalents'].iloc[0] 
    debt = stock.balance_sheet.loc['Long Term Debt'].iloc[0] 
    net_debt = debt - cash 

    return cash/1e9, debt/1e9, net_debt/1e9

def get_equity_value(ev, net_debt):
    return ev - net_debt

def get_implied_share_price(ticker, equity_value):
    stock= yf.Ticker(ticker)
    share_outstanding = stock.info['sharesOutstanding']/1e9
    per_share_value = equity_value / share_outstanding

    return share_outstanding, per_share_value

In [90]:
ticker = "KO"
print(f"WACC for {ticker}: {get_wacc(ticker)}")
rate = get_revenue_growth(ticker)
rate

WACC for KO: 0.0643


0.06834047751781329

In [92]:
wacc = get_wacc(ticker)

In [94]:
projection = get_revenue_projection(ticker=ticker, growth_rate= rate)

In [96]:
ebit_margin= get_ebit_margin(ticker)

In [98]:
ebit = get_ebit_projection(projection, ebit_margin)

In [100]:
NOPAT = get_NOPAT(ebit)

In [102]:
da_estimite = get_depreciation_and_amortization(ticker, projection)

In [104]:
capex = get_capex(ticker, projection)

In [106]:
nwc= get_workingcapital(ticker, projection)

In [108]:
fcf = get_fcf(NOPAT, da_estimite, capex, nwc)

In [110]:
pv_fcf = get_fcf_pv(fcf, wacc)

In [112]:
tv = get_terminal_value(fcf, wacc)

In [114]:
pv_tv = get_pv_tv(tv, wacc)

In [116]:
ev = get_enterprise_value(pv_fcf, pv_tv)

In [118]:
cash, debt, net_debt = get_net_debt(ticker)

In [120]:
equity_value = get_equity_value(ev, net_debt)

In [122]:
share_count, share_price = get_implied_share_price(ticker, equity_value)

In [124]:
df= pd.concat([projection, ebit, NOPAT, da_estimite, capex,nwc, fcf, pv_fcf ],
              keys=['Revenue', 'EBIT','NOPAT (EBIT-TAX)','D&A(+)','CapEX(-)', 'Change in NWC(-)','Unlevered FCF',
                    'Present Value of FCF'], axis=1).T

In [126]:
df2 = pd.DataFrame(data=[tv, pv_tv, ev, cash, debt, equity_value, share_count, share_price], 
          index=['Terminal Value', 'PV of Terminal Value', 'Enterprise Value', 
                 'Cash (+)', 'Debt (-)', 'Equity Value', 'Shares Outstanding',
                'Implied Share Price'], columns=['2025'])

In [128]:
df

,2025,2026,2027,2028,2029
Revenue,50.280000,53.710000,57.380000,61.310000,65.500000
EBIT,14.450000,15.440000,16.490000,17.620000,18.830000
NOPAT (EBIT-TAX),11.415500,12.197600,13.027100,13.919800,14.875700
D&A(+),1.437492,1.535555,1.640479,1.752837,1.872628
CapEX(-),1.938393,2.070626,2.212112,2.363621,2.525153
Change in NWC(-),2.505234,2.676136,2.858996,3.054811,3.263580
Unlevered FCF,8.409365,8.986393,9.596472,10.254205,10.959594
Present Value of FCF,7.901311,7.933362,7.960116,7.991821,8.025538


In [130]:
df2

,2025
Terminal Value,252.342796
PV of Terminal Value,184.786653
Enterprise Value,224.598802
Cash (+),10.828000
Debt (-),42.375000
Equity Value,193.051802
Shares Outstanding,4.303570
Implied Share Price,44.858526
